# Initialize
Run this section to call imports and run calibration. This section takes about 30 seconds to operate and requires input from the user.


In [1]:
import numpy as np
import cv2
from ScreenRemap import runCalibration

pts, frmMask = runCalibration()

Screen not found

	Threshold: 130


In [4]:
# define blob parameters, search
blobParams = cv2.SimpleBlobDetector_Params()
blobParams.filterByArea = True
blobParams.filterByColor = False
blobParams.filterByInertia = False
blobParams.filterByCircularity = False
blobParams.filterByConvexity = False
blobParams.minArea = 2000
blobParams.maxArea = 50000
myBlob = cv2.SimpleBlobDetector_create(blobParams)

# define auras look-up
myAuras = { 'One':None, 'Two':None, 'Water': None, 'Fire':None}
for select in myAuras.keys():
    imOver = cv2.imread(f".\\auras\\{select} Aura.png").astype(float)
    imAlpha = cv2.imread(f".\\auras\\{select} Aura - Alpha.png").astype(float)
    imAlpha = imAlpha / imAlpha.max()
    myAuras[select] = (imOver, imAlpha)

In [5]:
def drawAura(imDst, coords, select):
    global myAuras
    imSrc = myAuras[select][0]
    imAlpha = myAuras[select][1]

    # calculate positions
    Oy, Ox, _ = imSrc.shape
    top = int (coords[1] - (Oy / 2))
    bottom = top + Oy
    left = int (coords[0] - (Ox / 2))
    right = left + Ox

    print(f'\ttop:{top}\tbottom:{bottom}\n\tleft:{left}\tright:{right}')
    # too far up
    if top < 0:
        Iy_start = 0
        Oy_start = abs(top)
    else:
        Iy_start = top
        Oy_start = 0
    # too far down
    if bottom > imDst.shape[0]:
        Iy_stop = imDst.shape[0]
        Oy_stop = Oy - (bottom - Iy_stop)
    else:
        Iy_stop = bottom
        Oy_stop = Oy
    # too far left
    if left < 0:
        Ix_start = 0
        Ox_start = abs(left)
    else:
        Ix_start = left
        Ox_start = 0
    # too far right
    if right > imDst.shape[1]:  
        Ix_stop = imDst.shape[1]
        Ox_stop = Ox - (right - Ix_stop)
    else:
        Ix_stop = right
        Ox_stop = Ox

    imSubOver = imSrc[Oy_start:Oy_stop, Ox_start:Ox_stop]
    imSub = imDst[Iy_start:Iy_stop, Ix_start:Ix_stop].astype(float)
    if imAlpha is not None:
        imSubOver *= imAlpha[Oy_start:Oy_stop, Ox_start:Ox_stop]
        imSub *= 1 - imAlpha[Oy_start:Oy_stop, Ox_start:Ox_stop]
    imSub = cv2.add(imSubOver, imSub)
    imDst[Iy_start:Iy_stop, Ix_start:Ix_stop] = imSub.astype(np.uint8)

    return imDst

## Generate Homography
This section generates the homographic warp.

In [40]:
sW = 1280
sH = 720
winMain = 'Reference Screen'
winLive = 'Live View'

H, ret = cv2.findHomography(np.array(pts), np.array([[0, sH],[0, 0],[sW, 0],[sW, sH]]))

# generate base image
vid = cv2.VideoCapture(0)
imSrc = cv2.imread('background.png')
imOut = imSrc.copy()
cv2.imshow(winMain, imSrc)
cv2.waitKey(250)

-1

## Grab Frame

In [82]:
ret, frmCurr = vid.read()
if ret:
    frmCurr = cv2.warpPerspective(frmCurr, H, (sW, sH))
    frmDiff = imOut.copy()
    frmCurr = cv2.cvtColor(frmCurr, cv2.COLOR_BGR2GRAY)
    frmDiff = cv2.cvtColor(frmDiff, cv2.COLOR_BGR2GRAY)
    frmDiff = cv2.GaussianBlur(frmDiff, (17, 17), 0)
    cv2.subtract(frmDiff, frmCurr, frmDiff)
    frmDiff = cv2.GaussianBlur(frmDiff, (19, 19), 0)
    cv2.imshow('test',frmDiff)
    cv2.waitKey(100)

# Threshold
This code applies a few morphological transforms to resolve the difference between the reference image and the capture.
The results are presented as a grayscale difference. White is 

In [83]:
frmThres = frmDiff.copy()
frmThres = cv2.GaussianBlur(frmThres, (27, 27), 0)
cv2.multiply(frmThres, 255/frmThres.max(), frmThres)
frmThres = cv2.erode(frmThres,np.ones(19, np.uint8))
cv2.medianBlur(frmThres, 23, frmThres)
# display image
cv2.imshow('test',frmThres)
cv2.waitKey(100)

-1

# Object Analysis Test
This code uses blob analysis to identify discrete objects in the scene

In [84]:
from cmath import inf

frmTest = frmThres.copy()
_, frmTest = cv2.threshold(frmTest, 100, 255, cv2.THRESH_BINARY)
if frmTest.max() != frmTest.min():
    frmTest = frmTest.max() - frmTest
else:
    frmTest[:,:] = 255
# add border to allow on-edge blob detect
cv2.rectangle(frmTest, (0,0), (sW-1,sH-1), 255, 8)

keyPoints = myBlob.detect(frmTest)

frmTest = cv2.drawKeypoints(frmTest, keyPoints, np.array([]),
                            (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('test',frmTest)
cv2.waitKey(100)

-1

# Overlay Test
This section applies overlays to the base image where objects are detected

In [85]:
myPts = cv2.KeyPoint_convert(keyPoints)

try:
    print(f'Points found: {myPts.shape[0]}')
except:
    pass

imOut = imSrc.copy()
for idx, pt in enumerate(myPts):
    print(f'Point {idx:-3d}: {pt}')
    imOut = drawAura(imOut, pt, list(myAuras.keys())[idx % len(myAuras.keys())])

cv2.imshow(winMain, imOut)
cv2.waitKey(250)

Points found: 2
Point   0: [974.7595 490.9661]
	top:390	bottom:590
	left:924	right:1024
Point   1: [453.42896 220.8676 ]
	top:120	bottom:320
	left:403	right:503


-1